# Predicting the species of flower for a given input image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from  sklearn.metrics  import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
import re
import numpy as np
import cv2
import os
import imutils
import glob
import argparse
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from imutils import paths
import argparse

In [ ]:
Fimg = list(paths.list_images('/content/drive/MyDrive/502/datasets_final/flowers17/images'))

In [ ]:

class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.inter = inter
    def preprocess(self, image):
        return cv2.resize(image, (self.width, self.height),
            interpolation=self.inter)
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        self.preprocessors = preprocessors
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, imagePaths, verbose=-1):
        data = []
        labels = []

        for (i, imagePath) in enumerate(imagePaths):
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)
            data.append(image)
            labels.append(label)
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1,
                    len(imagePaths)))

        return (np.array(data), np.array(labels))

In [ ]:
#initialize the image preprocessor, 
fsp = SimplePreprocessor(32,32)
fsd = SimpleDatasetLoader(preprocessors=[sp])

(flowers, labels) = fsd.load(Fimg, verbose = 500)
flowers = flowers.reshape((flowers.shape[0], 3072))
 
 # showing some information on memory consumption of the images
print("INFO feature matrix: {:.1f}MB".format(flowers.nbytes/(1024*1000.0)))

In [ ]:
labels

In [ ]:
#Applying an Encoder to the labels of each flower
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels

In [ ]:
#Partioning the data into training adn testing splits using 20% for test images
#the remaining 80% for training data

Xtrain, Xtest, ytrain, ytest = train_test_split(flowers,labels,test_size=0.2, random_state=42)

In [ ]:

Xtrain_scale = Xtrain/255

In [ ]:
#Using logistic regression
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(Xtrain,ytrain)

y_pred = logistic_regression.predict(Xtest)

In [ ]:
print(classification_report(ytest,logistic_regression.predict(Xtest), target_names = le.classes_))

In [ ]:
plot_confusion_matrix(logistic_regression, Xtest, ytest) 
plt.title('Logistic Regression')
plt.show()

In [ ]:
#Using KNN Classifier
model = KNeighborsClassifier()
model.fit(Xtrain,ytrain)

In [ ]:
print(classification_report(ytest,model.predict(Xtest), target_names = le.classes_))

In [ ]:
plot_confusion_matrix(model, Xtest, ytest) 
plt.title('K-NN classifier')
plt.show()

In [ ]:
#Using Neural Network
flp = MLPClassifier(hidden_layer_sizes=(512,256,128,64),activation="relu",random_state=1)

In [ ]:
flp.fit(Xtrain, ytrain)
print(classification_report(ytest, mlp.predict(Xtest),target_names=le.classes_))

In [ ]:
plot_confusion_matrix(mlp, Xtest, ytest) 
plt.title('Neural Network')
plt.show()

In [ ]:
#Perform a K-means clustering
km = KMeans(n_clusters=5,init = 'k-means++')
y_pred = km.fit_predict(flowers)

In [ ]:
Error = []

for k in range(1,20,2):
  km1 = KMeans(n_clusters=k, init="k-means++")
  km1.fit(flowers)
  Error.append(km1.inertia_)

plt.figure(figsize=(12,6))    
plt.plot(range(1,20,2),Error, linewidth=2, color="red", marker ="8")
plt.xlabel("K Value")
plt.xticks(np.arange(1,20,2))
plt.ylabel("Error")
plt.show()

In [ ]:
#Create a new test dataset of 10 images
fsm_img = list(paths.list_images('/content/drive/MyDrive/502/datasets_final/jpg'))
fsm_path = np.array(fsm_img)

In [ ]:
f_sp = SimplePreprocessor(32,32)
f_sd = SimpleDatasetLoader(preprocessors=[f_sp])

(data, labels) = sd.load(fsm_path, verbose=500)
data = data.reshape((data.shape[0], 3072))

flowerLabels = ['bluebell', 'buttercup', 'coltsfoot', 'cowslip', 'crocus', 'daffodil', 'daisy', 'dandelion', 'fritillary', 'iris', 'lilyvalley', 'pansy', 'snowdrop', 
 'sunflower', 'tigerlily', 'tulip', 'windflower']

In [ ]:
from google.colab.patches import cv2_imshow

def display(predicted):
  f = [flowerLabels[a] for a in predicted]
  print(f'Flowers found : \n {f}\n')
  for (i, imagePath) in enumerate(image_path):
    # load the example image, draw the prediction, and display it
    # to our screen
    image = cv2.imread(imagePath)
    cv2.putText(image, "Label: {}".format(flowerLabels[predicted[i]]),(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2_imshow(image)
    cv2.waitKey(1)

In [ ]:
#Using Logistic Regression
print("Predicting with Logic Regression")
f1_pred = logistic_regression.predict(data)
f1_pred
display(f1_pred)

In [ ]:
#Using K-NN Classifier
print("Predicting with K-NN Classifier")
f2_pred = model.predict(data)
f2_pred
display(f2_pred)

In [ ]:
#Using Neural Network
print("Predicting with Neural Network")
f3_pred = mlp.predict(data)
f3_pred
display(f3_pred)